In [1]:
!wget 'https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0' positive.csv
!wget 'https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0' negative.csv

--2019-04-27 16:45:36--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.70.1
Подключение к www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 301 Moved Permanently
Адрес: /s/raw/fnpq3z4bcnoktiv/positive.csv [переход]
--2019-04-27 16:45:37--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc1acd8994049c7fa4078e9924d2.dl.dropboxusercontent.com/cd/0/inline/Af1brmj83-LYAaqYcP0uv_DGQu0ckSXb8ilhizfNCUTz7rK5Fhe7XDxshd1A67tWUN_-NpirS15dRlB8ufKJMrvYL8tO--MHR6PSfHhb5uhIBoJb7w3sAcQstx7ye7sp_Lk/file# [переход]
--2019-04-27 16:45:37--  https://uc1acd8994049c7fa4078e9924d2.dl.dropboxusercontent.com/cd/0/inline/Af1brmj83-LYAaqYcP0uv_DGQu0ckSXb8ilhizfNCUTz7rK5Fhe7XDxshd1A67tWUN_-NpirS15dRlB8ufKJMrvYL8tO--MHR6PSfHhb5uhIBoJb7w3sAcQstx7y

In [2]:
import pandas as pd

In [3]:
positive = pd.read_csv('positive.csv?dl=0',header=None,encoding='utf-8',delimiter=';')
negative = pd.read_csv('negative.csv?dl=0',header=None,encoding='utf-8',delimiter=';')

In [4]:
positive=positive[[positive.columns[3]]]
negative=negative[[negative.columns[3]]]

In [5]:
positive['target']=1
negative['target']=0

In [6]:
positive.columns=['text','target']
negative.columns=['text','target']

In [7]:
positive.head()

,text,target
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


In [8]:
full_set = pd.concat([positive,negative])

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [10]:
x_train,x_test,y_train,y_test = train_test_split(full_set['text'],full_set.target.values,test_size=0.2,shuffle=True,random_state=2121)

In [11]:
clf = make_pipeline(TfidfVectorizer(ngram_range=(1,2)),MultinomialNB())
clf.fit(x_train,y_train)
preds = clf.predict(x_test)

In [12]:
accuracy_score(y_test,preds)

0.7739546366301496

## Теперь опробуем на этих же данных простейшую сеть с предобученными ембедингами

In [13]:
!wget http://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.vec

--2019-04-27 16:46:42--  http://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.vec
Распознаётся files.deeppavlov.ai (files.deeppavlov.ai)… 93.175.29.74
Подключение к files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 2767469831 (2,6G) [application/octet-stream]
Сохранение в: «ft_native_300_ru_twitter_nltk_word_tokenize.vec»

ft_native_300_ru_tw 100%[===================>]   2,58G  2,56MB/s    за 17m 56s 

2019-04-27 17:04:38 (2,45 MB/s) - «ft_native_300_ru_twitter_nltk_word_tokenize.vec» сохранён [2767469831/2767469831]



In [14]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding
from tqdm import tqdm_notebook

Using TensorFlow backend.


In [15]:
f = open('ft_native_300_ru_twitter_nltk_word_tokenize.vec')
embedding_values = {}
for line in tqdm_notebook(f):
    value = line.replace('\n','').split(' ')
    word = value[0]
    coef = np.array(value[1:-1],dtype = 'float32')
    embedding_values[word]=coef

A Jupyter Widget

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(full_set['text'])
sequences_train = tokenizer.texts_to_sequences(x_train)
sequences_test = tokenizer.texts_to_sequences(x_test)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 291376 unique tokens.


In [17]:
embedding_matrix = np.zeros((len(word_index)+1, 100),dtype=np.float32)
sum_finding=0
for word, i in tqdm_notebook(word_index.items()):
    try:
        embedding_vector = embedding_values[word]
        sum_finding=sum_finding+1
    except:
        embedding_vector = embedding_values["unknown"]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

A Jupyter Widget

In [18]:
print('Нашли векторов - ',sum_finding)

Нашли векторов -  147597


In [19]:
from __future__ import print_function
import numpy as np
np.random.seed(1337) 

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding, SpatialDropout1D
from tensorflow.keras.layers import LSTM, SimpleRNN, GRU, Bidirectional
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import RMSprop,Adam

max_features = 20000
maxlen = 40  
batch_size = 256 

print('Готовим модель...')
model = Sequential()
model.add(Embedding(
    len(word_index)+1,
    100,
    weights=[embedding_matrix],
    input_length=maxlen,
    trainable=False
))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(GRU(32)))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-3,clipnorm=5,clipvalue=5),
              metrics=['accuracy'])

X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)

print('Обучаем...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=25,
          validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Тест score:', score)
print('Тест accuracy:', acc)

Готовим модель...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Обучаем...
Train on 181467 samples, validate on 45367 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/25
181467/181467 [==============================] - 62s 341us/sample - loss: 0.5614 - acc: 0.7033 - val_loss: 0.5113 - val_acc: 0.7413
Epoch 2/25
181467/181467 [==============================] - 60s 330us/sample - loss: 0.5122 - acc: 0.7395 - val_loss: 0.4831 - val_acc: 0.7600
Epoch 3/25
181467/181467 [==============================] - 60s 331us/sample - loss: 0.4947 - acc: 0.7505 - val_loss: 0.4886 - val_acc: 0.7518
Epoch 4/25
181467/181467 [==============================] - 59s 326us/sample - loss: 0.4834 - acc: 0.7590 - val_loss: 0.4775 - val_acc: 0.7604
Epoch 5/25
181467/181467 [==============================] - 60s 330us/sample - loss: 0.4748 - acc: 0.7650 - val_l